In [106]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import urllib.request
from bs4 import BeautifulSoup
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Part 1

## Initial scrapping. Pull the URL, add it to a page object.I have decided not to display the objects to simplify the notebook

In [107]:
# Wikipedia page containing th neighborhoods I'll be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# Beautiful soup import completed in the impor section
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

## The table has been identified to use a wikitable class. Create an object containing only that table

In [108]:
#The table of interest has a 'wikitable' class defined. statement below pulls all objects with that class. Second statement display content found
neighborhood_table=soup.find('table', class_='wikitable')

## Create lists to contain each column. Iterate through the created able to fill the arrays. Although in this version of the Wikipedia table there are no empty neighborhoods with an assigned borough. The logic to comply with the rubric is added in the second loop 

In [109]:
#Add each column into an list. The resulting arrays will then be converted to a DataFrame
A=[]
B=[]
C=[]
for row in neighborhood_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
for n in range (0, len(C)):
    if C[n]=="\n":
        C[n]=B[n]

## Create the dataframe. Eliminate line breaks. Drop rows where Borough is Not Assigned. Display the dataframe

In [113]:
#Create a dataframe with the resulting arrays. Clear line breaks, remove not assigned Postal Codes
df = pd.DataFrame({'Postal Code':A,'Borough': B, 'Neighborhood':C})
df
df = df.replace(r'\n','', regex=True)
df = df.replace(r'/',',', regex=True)
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


## Display shape as per rubric

In [111]:
df.shape

(103, 3)

# Part 2

## Import csv with Latitude and Longitude. Decided to go this route due to inconsistencies with the geocoder package 

In [112]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [115]:
df3 = pd.merge(df,df2, on ="Postal Code")
df3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
